In [1]:
# UNCOMMENT THESE TO INSTALL THE REQUIRED LIBRARIES

# %pip install pandas
# %pip install tensorflow
# %pip install scikit-learn
# %pip install pymysql
# %pip install sshtunnel
# %pip install keras

In [1]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import pymysql
import sshtunnel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Sklearn version:", sklearn.__version__)
print("Pymysql version:", pymysql.__version__)
print("Sshtunnel version:", sshtunnel.__version__)

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Set up SSH tunnel
server = sshtunnel.SSHTunnelForwarder(
    ('selene.hud.ac.uk', 22),
    ssh_username='workit',
    ssh_password='umbra(despair>Quartz218',
    remote_bind_address=('localhost', 3306)
)
server.start()

# Defining the connection
connection = pymysql.connect(host='localhost', port=server.local_bind_port, user='workit', password='umbra(despair>Quartz218', database='workit')

In [ ]:
# DATAPATH = "../input/megaGymDataset.csv"

# Load the dataset
dataset = pd.read_sql_query("SELECT * FROM `megaGymDataset`", connection)
dataset.head()

In [ ]:
server.stop()

In [ ]:
def labelEncodersScaler(data):
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return label_encoders, scaler

In [6]:
def train_model(data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Build the ANN model using TensorFlow
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(data["ID"].unique()), activation='softmax')  # Use the unique number of IDs as the output dimension
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    return model

In [ ]:
def recommend_exercises(user_input, model, data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

In [ ]:
model = train_model(dataset.copy())
model.save("ann_gym_recommender.keras")

In [ ]:
model = load_model(MODEL_PATH)
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input, model, dataset.copy())
dataset.iloc[IDs]